# Caffe Neural Net Tutorial

This is a Notebook execution of the Caffe Tutorial:
http://christopher5106.github.io/deep/learning/2015/09/04/Deep-learning-tutorial-on-Caffe-Technology.html

### Load Training Model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import caffe
from IPython.display import Image as I_Image

In [11]:
import os

os.environ

{'TMP': 'C:\\Users\\Dallas\\AppData\\Local\\Temp', 'COMPUTERNAME': 'DESKTOP-NIOF01C', 'VS140COMNTOOLS': 'C:\\Program Files (x86)\\Microsoft Visual Studio 14.0\\Common7\\Tools\\', 'VS110COMNTOOLS': 'C:\\Program Files (x86)\\Microsoft Visual Studio 11.0\\Common7\\Tools\\', 'USERDOMAIN': 'DESKTOP-NIOF01C', 'PSMODULEPATH': 'C:\\Program Files\\WindowsPowerShell\\Modules;C:\\WINDOWS\\system32\\WindowsPowerShell\\v1.0\\Modules', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'CUDA_PATH_V8_0': 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v8.0', 'PROCESSOR_IDENTIFIER': 'Intel64 Family 6 Model 94 Stepping 3, GenuineIntel', 'VBOX_MSI_INSTALL_PATH': 'C:\\Program Files\\Oracle\\VirtualBox\\', 'PROGRAMFILES': 'C:\\Program Files', 'PROCESSOR_REVISION': '5e03', 'NVCUDASAMPLES8_0_ROOT': 'C:\\ProgramData\\NVIDIA Corporation\\CUDA Samples\\v8.0', 'SYSTEMROOT': 'C:\\WINDOWS', 'CLICOLOR': '1', 'PROGRAMFILES(X86)': 'C:\\Program Files (x86)', 'TK_LIBRARY': 'C:\\Users\\Dallas\\Anaconda2\\

In [5]:
caffe.set_mode_gpu()

In [ ]:
#Build neural net from prototext file
net = caffe.Net('sample.prototxt', caffe.TEST)

In [ ]:
print net.inputs

# Display out layers with respective shapes
[(k, v.data.shape) for k, v in net.blobs.items()]

In [ ]:
# Display weight & bias parameters
[(k, v[0].data.shape, v[1].data.shape) for k, v in net.params.items()]

In [ ]:
print net.blobs['conv'].data.shape

### Draw Neural Network

In [ ]:
# Draw script: C:\Users\Dallas\DataScience\caffe-master\Build\x64\Release\pycaffe\draw_net.py
# Net file : C:\Users\Dallas\OneDrive\MSBA Program\Fall 2016 Courses\Adv Predictive Modeling\APM-Project\Code\sample.prototxt
# Net file: C:\Program Files\caffe-master\examples\net_surgery\conv.prototxt

! python "C:\Users\Dallas\DataScience\caffe-master\Build\x64\Release\pycaffe\draw_net.py"\
         "C:\Users\Dallas\OneDrive\MSBA Program\Fall 2016 Courses\Adv Predictive Modeling\APM-Project\Code\sample.prototxt"\
          my_net.png

I_Image(filename = "my_net.png")

### Load & Prepare Training Data

In [ ]:
im = np.array(Image.open('C:\Users\Dallas\DataScience\caffe-master\examples\images\cat_gray.jpg'))
plt.imshow(im,cmap = 'gray')
plt.show()

In [ ]:
#Reshape the data blob (1, 1, 100, 100) to the new size (1, 1, 360, 480) to fit the image
im_input = im[np.newaxis, np.newaxis, :, :]
net.blobs['data'].reshape(*im_input.shape)
net.blobs['data'].data[...] = im_input

net.blobs['data'].data.shape

In [ ]:
#Compute blobs given input
net.forward()

In [ ]:
#Print convolutions
for i in range(3):
    plt.subplot(1, 3, i+1), plt.imshow(net.blobs['conv'].data[0,i], cmap = 'gray')

plt.show()

### Download & Run Pre-Trained Model

In [ ]:
# Download Model script: C:/Program Files/caffe-master/scripts/download_model_binary.py
# Download Model script from Gist: C:/Program Files/caffe-master/scripts/download_model_from_gist.sh
# Script location for model of interest: C:/Program Files/caffe-master/data/ilsvrc12/get_ilsvrc_aux.sh
# Location to save pre-trained model: C:/Program Files/caffe-master/models/bvlc_reference_caffenet

! python "C:/Users/Dallas/DataScience/caffe-master/scripts/download_model_binary.py" "C:/Users/Dallas/DataScience/caffe-master/models/bvlc_reference_caffenet" 

In [ ]:
# Draw file location: "C:\Program Files\caffe-master\Build\x64\Release\pycaffe\draw_net.py"
# Net file location: "C:\Program Files\caffe-master\examples\net_surgery\conv.prototxt"

! python "C:\Users\Dallas\DataScience\caffe-master\Build\x64\Release\pycaffe\draw_net.py"\
         "C:\Users\Dallas\DataScience\caffe-master\examples\net_surgery\bvlc_caffenet_full_conv.prototxt"\
          my_net2.png

I_Image(filename = "my_net2.png")

In [ ]:
# Build Neural Net w/ Pre-Trained Model
net = caffe.Net('C:/Users/Dallas/DataScience/caffe-master/models/bvlc_reference_caffenet/deploy.prototxt',
                'C:/Users/Dallas/DataScience/caffe-master/models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
                caffe.TEST)

# load data input and configure preprocessing
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_mean('data', np.load('python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1))
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

# Change expected batch size from 10 to 1 since only classifying one image
net.blobs['data'].reshape(1,3,227,227)

#load the image in the data layer
im = caffe.io.load_image('C:\Users\Dallas\DataScience\caffe-master\examples\images\cat.jpg')
net.blobs['data'].data[...] = transformer.preprocess('data', im)

#compute
out = net.forward()

#predicted predicted class
print out['prob'].argmax()

#print predicted labels
labels = np.loadtxt("C:/Users/Dallas/DataScience/caffe-master/data/ilsvrc12/synset_words.txt", str, delimiter='\t')
top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
print labels[top_k]

## Train Model from Scratch

In [ ]:
solver = caffe.get_solver('C:/Users/Dallas/DataScience/caffe-master/models/bvlc_reference_caffenet/solver.prototxt')